#데이터 LOAD

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -qq "/content/drive/MyDrive/넥스트랩/dataset/0908_RealLast300.zip" #개인경로로 바꾸기


In [ ]:
!pip install -U git+https://github.com/leondgarse/keras_efficientnet_v2
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras_efficientnet_v2 as efficientnet_v2
from tensorflow.keras.layers import Dense, Input, Conv2D, Dropout, Flatten, Activation, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model, Sequential
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image



#Image Generator

In [ ]:
train_path = '/content/split/train'
val_path = '/content/split/valid'

Batch_size = 64
Input_size = (240,240)
np.random.seed(27)
epochs = 50
num_classes = 322
input_shape = (240,240,3)

train_generator = ImageDataGenerator(
                                      rescale=1/255.0,
                                     zoom_range = 0.2
                                     )
valid_generator = ImageDataGenerator(rescale=1/255.0)

test_datagen = ImageDataGenerator(rescale=1./255)


flow_train_gen = train_generator.flow_from_directory(train_path,
                                                  class_mode = 'categorical',
                                                  batch_size=Batch_size ,
                                                  target_size =Input_size,
                                                  shuffle=True)

flow_val_gen = valid_generator.flow_from_directory(val_path,
                                                   class_mode = 'categorical',
                                                   batch_size=Batch_size, 
                                                   target_size= Input_size,
                                                   shuffle=False)

Found 77280 images belonging to 322 classes.
Found 19320 images belonging to 322 classes.


In [ ]:
# steps 횟수를 구하기 위해 학습 데이터의 건수와 검증 데이터의 건수를 구함. steps = ceil(학습 데이터 건수/BATCH_SIZE)
# 파일개수는 제너레이터 아래 기록된 path
train_paths = 77280
val_paths = 19320
steps_per_epoch = int(np.ceil(train_paths / Batch_size))
validation_steps = int(np.ceil(val_paths/ Batch_size))



#모델구축

In [ ]:
def create_model(version = 'v1', model_type='b0', in_shape= (224,224,3), n_classes = 322):
 
  if version == 'v1':
    if model_type == 'b0':
      base_model = efficientnet_v2.EfficientNetV1(model_type, input_shape=input_shape, num_classes=0, include_preprocessing=False,  pretrained='imagenet')
    elif model_type == 'b1':
      base_model = efficientnet_v2.EfficientNetV1(model_type, input_shape=input_shape, num_classes=0,include_preprocessing=False, pretrained='imagenet')
  
  elif version == 'v2':
    if model_type == 'b0':
      base_model = efficientnet_v2.EfficientNetV2(model_type, input_shape=input_shape, num_classes=0, include_preprocessing=False, pretrained='imagenet')
    elif model_type == 'b1':
      base_model = efficientnet_v2.EfficientNetV2(model_type, input_shape=input_shape, num_classes=0, include_preprocessing=False, pretrained='imagenet')

  for layer in base_model.layers[:-30]: # just exclude last layer from copying
    layer.trainable = False

  model = Sequential()
  model.add(Input(shape=in_shape))
  model.add(base_model)
  model.add(GlobalAveragePooling2D())
  model.add(Dropout(0.2))
  model.add(Dense(n_classes, activation='softmax'))
  
  return model

# EfficientNetV2 모델 생성. 

In [ ]:
# callback
save_dir = '/content/drive/MyDrive/' #개인경로
checkpoint_name = f'0908_{epochs}-batch-{Batch_size}_B1_300.h5' #개인 save이름
save_path = os.path.join(save_dir,checkpoint_name)

checkpoint = ModelCheckpoint(save_path,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='auto'
                            )

reduceLR = ReduceLROnPlateau(monitor='val_loss',
                             factor=0.8,
                             patience=3,
                             min_lr = 0.01* 0.8 *0.8 * 0.8 * 0.8
                             )

earlystopping = EarlyStopping(monitor='val_loss',
                              patience=10,
                             )

In [ ]:
top3_acc = tf.keras.metrics.TopKCategoricalAccuracy(
    k=3, name="top3_acc", dtype=None
)
model = create_model(version = 'v2', model_type='b1', in_shape= input_shape, n_classes = num_classes)

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['acc',top3_acc])
model.summary()

In [ ]:
history = model.fit(flow_train_gen, 
                    epochs=epochs, 
                    steps_per_epoch = steps_per_epoch,
                    validation_data = flow_val_gen, 
                    validation_steps = validation_steps,
                    callbacks = [earlystopping, checkpoint, reduceLR])

In [ ]:
model_name = '0908_nonfreeze_last300' #각자 정하고 싶은 이름
model_path = os.path.join(save_dir,model_name)
model.save(model_path)

In [ ]:
weight_path = os.path.join(model_path,'0908_nonfreeze_last300_weight')
model.save_weights(weight_path)

#모델 시각화


In [ ]:
#Plotting 
acc = history.history['acc'] 
val_acc = history.history['val_acc'] 
loss = history.history['loss'] 
val_loss = history.history['val_loss'] 

plt.figure(figsize=(8, 8)) 

plt.subplot(2, 1, 1) 
plt.plot(acc, label='Training Accuracy') 
plt.plot(val_acc, label='Validation Accuracy') 
plt.legend(loc='lower right') 
plt.ylabel('Accuracy') 
plt.ylim([min(plt.ylim()),1.0]) 
plt.title('Training and Validation Accuracy') 

plt.subplot(2, 1, 2) 
plt.plot(loss, label='Training Loss') 
plt.plot(val_loss, label='Validation Loss') 
plt.legend(loc='upper right') 
plt.ylabel('Cross Entropy') 
plt.ylim([0,10]) 
plt.title('Training and Validation Loss') 
plt.xlabel('epoch') 
plt.show()


In [ ]:
#ImageDataGenerator를 통해서 오토-레이블링 된 class를 가져옵니다.
classes_dict = flow_train_gen.class_indices
classes_dict = dict(map(reversed, classes_dict.items()))

def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(224,224))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model.predict([x])[0]

    result = list(result)
    
    classname_list = []
    pred_value_list = []
    for _ in range(3) :
      index= result.index(max(result))
      classname_list.append(classes_dict[index])
      pred_value_list.append(max(result))
      result[index] = 0.

    return classname_list, pred_value_list

###모델 테스트

In [ ]:
test_car1_path = '/content/drive/MyDrive/넥스트랩/test_sample_image/MAH02950_1.jpg2.jpg' #test하고 싶은 image
classnames1, values1 = predict_image(test_car1_path)

for index, (classname, value) in enumerate(zip(classnames1, values1)) :
  print(f'[{index}] class : {classname}\npredict value : {value}\n\n')

img = image.load_img(test_car1_path)
plt.imshow(img)

In [ ]:
test_car2_path = '/content/drive/MyDrive/넥스트랩/test_sample_image/MAH02950_793.jpg1.jpg' #test하고 싶은 image
classnames2, values2 = predict_image(test_car2_path)

for index, (classname, value) in enumerate(zip(classnames2, values2)) :
  print(f'[{index}] class : {classname}\npredict value : {value}\n\n')

img = image.load_img(test_car2_path)
plt.imshow(img)

In [ ]:
classnames3, values3 = predict_image(test_car3_path)
test_car3_path = '/content/drive/MyDrive/넥스트랩/test_sample_image/MAH02950_908.jpg3.jpg' #test하고 싶은 image

for index, (classname, value) in enumerate(zip(classnames3, values3)) :
  print(f'[{index}] class : {classname}\npredict value : {value}\n\n')

img = image.load_img(test_car3_path)
plt.imshow(img)

In [ ]:
test_car4_path = '/content/drive/MyDrive/넥스트랩/test_sample_image/MAH02950_561.jpg2.jpg' #test하고 싶은 image

classnames4, values4 = predict_image(test_car4_path)

for index, (classname, value) in enumerate(zip(classnames4, values4)) :
  print(f'[{index}] class : {classname}\npredict value : {value}\n\n')

img = image.load_img(test_car4_path)
plt.imshow(img)

In [ ]:
test_car4_path = '/content/drive/MyDrive/넥스트랩/test_sample_image/MAH02950_260.jpg0.jpg' #test하고 싶은 image
classnames4, values4 = predict_image(test_car4_path)

for index, (classname, value) in enumerate(zip(classnames4, values4)) :
  print(f'[{index}] class : {classname}\npredict value : {value}\n\n')

img = image.load_img(test_car4_path)
plt.imshow(img)

###히트맵 검증

In [ ]:
def make_gradcam_heatmap(img_array, model, pre_trained,last_conv_layer_name, classifier_layer_names):
    
    # First, we create a model that maps the input image to the activations
    # of the last conv layer
    last_conv_layer  = model.get_layer(pre_trained).get_layer(last_conv_layer_name)
    conv_model       = keras.Model(model.get_layer(pre_trained).inputs, last_conv_layer.output)
    # Second, we create a model that maps the activations of the last conv
    # layer to the final class predictions
    classifier_input = keras.Input(shape=last_conv_layer.output.shape[1:])
    x = classifier_input
    
    for layer_name in classifier_layer_names:
        x = model.get_layer(layer_name)(x)

    classifier_model = keras.Model(classifier_input, x)
    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer  
    with tf.GradientTape() as tape:
        # Compute activations of the last conv layer and make the tape watch it
        last_conv_layer_output = conv_model(img_array)
        tape.watch(last_conv_layer_output)
        
        # Compute class predictions
        preds = classifier_model(last_conv_layer_output)
        top_pred_index = tf.argmax(preds[0])
        top_class_channel = preds[:, top_pred_index]

    # This is the gradient of the top predicted class with regard to
    # the output feature map of the last conv layer
    grads = tape.gradient(top_class_channel, last_conv_layer_output)
    
    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    last_conv_layer_output = last_conv_layer_output.numpy()[0]

    # is our saliency heatmap of class activation
    saliency = np.mean(last_conv_layer_output, axis=-1)
    saliency = np.maximum(saliency, 0) / np.max(saliency)

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    pooled_grads = pooled_grads.numpy()
    
    for i in range(pooled_grads.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]

    # The channel-wise mean of the resulting feature map
    # is our grad_cam heatmap of class activation
    grad_cam = np.mean(last_conv_layer_output, axis=-1)
    grad_cam = np.maximum(grad_cam, 0) / np.max(grad_cam)

    return grad_cam, saliency

In [ ]:
def merge_with_heatmap(original_img, heatmap):
    original_img = np.array(original_img)
    resized_heatmap=resize(heatmap, (240, 240))
    resized_heatmap = np.uint8(255*resized_heatmap)
    resized_heatmap = cv2.applyColorMap(resized_heatmap, cv2.COLORMAP_JET)
    #resized_heatmap = cv2.cvtColor(resized_heatmap, cv2.COLOR_RGB2BGR)
    return cv2.addWeighted(resized_heatmap, 0.7, original_img, 0.5, 6)

def convert_to_heatmap(heatmap):
    heatmap = np.uint8(255*heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    return cv2.cvtColor(heatmap, cv2.COLOR_RGB2BGR)

In [ ]:
def show_hotmap (img, heatmap, title='Heatmap', alpha=0.6, cmap='jet', axisOnOff='off'):
    '''
    img     :    Image
    heatmap :    2d narray
    '''
    resized_heatmap=resize(heatmap, img.size)
    
    fig, ax = plt.subplots()
    ax.imshow(img)
    ax.imshow(resized_heatmap, alpha=alpha, cmap=cmap)
    plt.axis(axisOnOff)
    plt.title(title)
    plt.show()

In [ ]:
def prepare_single_input(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img = image.img_to_array(img)
    img /= 255.
    img = np.expand_dims(img, axis= 0) # (1, 224, 224, 3)
    return img

In [ ]:
def predict_image(Mymodel, img_path, top_k_num = 3):
    image = prepare_single_input(img_path, target_size = (240, 240))
    result = Mymodel.predict([image])[0]

    result = list(result)
    
    classname_list = []
    pred_value_list = []
    for _ in range(top_k_num) :
      index= result.index(max(result))
      classname_list.append(classes_dict[index])
      pred_value_list.append(max(result))
      result[index] = 0.

    return classname_list, pred_value_list

In [ ]:
######################################################################################
#defalut classifier_layer_names #기본 classifier_layer입니다.
#classifier_layer_names =  ['global_average_pooling2d', 'dropout', 'dense']
classifier_layers =  model.layers[-3:]
classifier_layer_names = []
for layer in classifier_layers:
    classifier_layer_names.append(layer.name)
last_conv_layer_name   = 'post_swish'
pre_train= 'EfficientNetV2'
Top_K = 1

true_cnt = 0
false_cnt = 0

#####################################################################################
image_paths = glob.glob('./data/test/dataset/**/*.jpg') #경로 설정해주기
for path in tqdm(image_paths, total = len(image_paths)) :
    img = Image.open(path).resize(size=input_shape[:2])  
    img_array = prepare_single_input(path, input_shape[:2])
    #test
    grad_cam, saliency = make_gradcam_heatmap(img_array, model, pre_train,last_conv_layer_name, classifier_layer_names)
    pred_classnames, pred_values = predict_image(model, path)


    grad_cam_merge = merge_with_heatmap(img, grad_cam)
    saliency_merge = merge_with_heatmap(img, saliency)

    grad_cam = convert_to_heatmap(grad_cam)
    saliency = convert_to_heatmap(saliency)

    #save

    #./data/test/dataset\BMW_SUV_X5_2019-2020\MAH02945_1017.jpg1.jpg -> BMW_SUV_X5_2019-2020
    true_classname = path.split('\\')[1]

    for index, pred_classname in enumerate(pred_classnames) :
        #BMW/SUV_X5/2019-2020 -> BMW_SUV_X5_2019-2020 True_class와 맞추기 위해서.
        pred_classnames[index] = pred_classname.replace('/','_')

    #pred_values안에 0.1이 있다면 True 그렇지 않다면 False인 리스트 생성
    flag_pred_value = list(map(lambda x : True if x >= 0.1 else False, pred_values))

    #실제 클래스와 예측 클래스가 일치하면, 그리고 predict_value가 0.1이상이라면 실행
    #Select save path True or False
    if (true_classname in pred_classnames) & (True in flag_pred_value) : 
        true_cnt+=1
        #./data/test/dataset\BMW_SUV_X5_2019-2020\MAH02945_1017.jpg1.jpg -> ./data/test/result/true/MAH02945_1017.jpg1.jpg
        split_paths = path.split('\\')
        save_path = split_paths[0].replace('dataset','result') +'/' + 'true' + '/' + split_paths[-1]
    else : 
        false_cnt+=1
        #./data/test/dataset\BMW_SUV_X5_2019-2020\MAH02945_1017.jpg1.jpg -> ./data/test/result/false/MAH02945_1017.jpg1.jpg
        split_paths = path.split('\\')
        save_path = split_paths[0].replace('dataset','result') +'/' + 'false' + '/' + split_paths[-1]

    #Save original image 
    img.save(save_path)
    #./data/test_result/true/MAH02945_1017.jpg1.jpg -> ./data/test_result/true or false/MAH02945_1017.jpg1.txt
    tmp_path = save_path[:-3] + 'txt'
    with open(tmp_path, "w", encoding="UTF-8") as f:
        f.write(f"[실제 클래스]\n")
        f.write(f"{true_classname}\n")
        f.write(f"[예측 클래스 Top 3]\n")
        f.write(f"{pred_classnames[0]}, 예측률 : {pred_values[0]}\n")
        f.write(f"{pred_classnames[1]}, 예측률 : {pred_values[1]}\n")
        f.write(f"{pred_classnames[2]}, 예측률 : {pred_values[2]}\n")

    #Save grad_cam image
    #./data/test_result/true/MAH02945_1017.jpg1.jpg -> ./data/test_result/true or false/MAH02945_1017.jpg1_gc.jpg
    tmp_path = save_path[:-4] + '_gc.jpg'
    plt.figure(figsize=(input_shape[0] / 100, input_shape[0] / 100))
    plt.imshow(grad_cam)
    plt.axis('off'), plt.xticks([]), plt.yticks([])
    plt.tight_layout()
    plt.subplots_adjust(left = 0, bottom = 0, right = 1, top = 1, hspace = 0, wspace = 0)
    plt.savefig(tmp_path, bbox_inces='tight', pad_inches=0, dpi=100)
    plt.close()
    

    #Save saliency image
    #./data/test_result/true/MAH02945_1017.jpg1.jpg -> ./data/test_result/true or false/MAH02945_1017.jpg1_sa.jpg
    tmp_path = save_path[:-4] + '_sa.jpg'
    plt.figure(figsize=(input_shape[0] / 100, input_shape[0] / 100))
    plt.imshow(saliency)
    plt.axis('off'), plt.xticks([]), plt.yticks([])
    plt.tight_layout()
    plt.subplots_adjust(left = 0, bottom = 0, right = 1, top = 1, hspace = 0, wspace = 0)
    plt.savefig(tmp_path, bbox_inces='tight', pad_inches=0, dpi=100)
    plt.close()
    

    #Save grad_cam-Merge image
    #./data/test_result/true/MAH02945_1017.jpg1.jpg -> ./data/test_result/true or false/MAH02945_1017.jpg1_gcm.jpg
    tmp_path = save_path[:-4] + '_gcm.jpg'
    cv2.imwrite(tmp_path, grad_cam_merge)

    #Save saliency-Merge image
    #./data/test_result/true/MAH02945_1017.jpg1.jpg -> ./data/test_result/true or false/MAH02945_1017.jpg1_sam.jpg
    tmp_path = save_path[:-4] + '_sam.jpg'
    cv2.imwrite(tmp_path, saliency_merge)

100%|██████████| 836/836 [04:04<00:00,  3.42it/s]


In [ ]:
acc = true_cnt / (true_cnt+false_cnt)
print(f"Top{Top_K} Acc : {round(acc,2)}")